# Create a forward operator and display sensitivity maps

`
Authors: Eric Larson <larson.eric.d@gmail.com>
         Denis Engemann <denis.engemann@gmail.com>
         Alex Gramfort <alexandre.gramfort@inria.fr>
`

License: BSD (3-clause)

First setup some paths. We will use the MNE sample data.

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import mne

# Change the following path to where the folder ds000117 is on your disk
data_path = os.path.expanduser("~/Documents/teaching/practical_meeg_2022_data/ds000117")

epochs_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss-epo.fif')

In [ ]:
from IPython.display import Image
from mayavi import mlab

In [ ]:
# to check that things work:
# mlab.test_contour3d()

# Computing the forward operator

To compute a forward operator we need:

   - a ``-trans.fif`` file that contains the coregistration info
   - a source space
   - the BEM surfaces

## Compute and visualize BEM surfaces


Here, we work with a **pre-computed BEM**. 

Computing the BEM surfaces requires FreeSurfer (get it at: https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall#Download) and makes use of either of the two following command line tools:

[mne watershed_bem](http://martinos.org/mne/dev/generated/commands.html#mne-watershed-bem)

[mne flash_bem](http://martinos.org/mne/dev/generated/commands.html#mne-flash-bem)

Or can be done by directly by calling the functions (FreeSurfer installation needed):

https://mne.tools/stable/generated/mne.bem.make_watershed_bem.html

https://mne.tools/stable/generated/mne.bem.make_flash_bem.html





If you would like to try out BEM reconstruction for yourself later, you can install FreeSurfer on MacOS or Linux directly and in a VirtualMachine for Windows. 

To compute BEM models for this dataset,  I set up the `SUBJECTS_DIR` and ran:

    mne watershed_bem -s sub-01 --overwrite
    mne make_scalp_surfaces -s sub-01 --force --overwrite

So, let's first look at the BEM surfaces.

For EEG we use 3 layers (inner skull, outer skull, and skin), while for MEG 1 layer (inner skull) is enough.


In [8]:
t1_fname = os.path.join(data_path, 'derivatives/freesurfer/sub-01/ses-mri/anat/mri/T1.mgz')

In [9]:
from nilearn import plotting
plotting.plot_anat(t1_fname);
plt.show()

In [16]:
subjects_dir = os.path.join(data_path, 'derivatives/freesurfer/sub-01/ses-mri/anat/mri')

In [18]:
mne.viz.plot_bem(subject='sub-01', subjects_dir=subjects_dir,
                 mri=t1_fname,
                 orientation='coronal');

OSError: Subject bem directory "/Users/brittawe/Documents/teaching/practical_meeg_2022_data/ds000117/derivatives/freesurfer/sub-01/ses-mri/anat/mri/sub-01/bem" does not exist

In [ ]:
mne.gui.coregistration(subject='sub-01', subjects_dir=subjects_dir, inst=raw_fname);

### Visualizing the coregistration

The coregistration is the operation that allows to position the head and the sensors in a common coordinate system. In the MNE software, the transformation to align the head and the sensors in stored in a so called *trans* file. It is a FIF file that ends with `-trans.fif`. It can be obtained with ``mne_analyze`` (Unix tools), ``mne.gui.coregistration`` (in Python) or mrilab if you're using a Neuromag system.

For the Python version, see https://mne.tools/dev/generated/mne.gui.coregistration.html

Here, we assume the coregistration is done, so we just visually check the alignment with the following code.

In [ ]:
trans_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-trans.fif')
info = mne.io.read_info(raw_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject='sub-01', dig=True,
                             subjects_dir=subjects_dir, verbose=True);

In [ ]:
mlab.close('all')

## Compute Source Space

The source space defines the position of the candidate source locations.

The following code computes such a source space with an OCT-6 resolution.

In [ ]:
mne.set_log_level('WARNING')
subject = 'sub-01'
src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

In [ ]:
src

``src`` contains two parts, one for the left hemisphere (4098 locations) and one for the right hemisphere (4098 locations).

In [ ]:
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=False, src=src,
                             subjects_dir=subjects_dir, verbose=True, meg=False,
                             eeg=False);

In [ ]:
mne.viz.plot_alignment(info, trans_fname, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

In [ ]:
mlab.close()

### Compute forward solution

We can now compute the forward solution.

To reduce computational load, we'll just compute a single layer BEM
(just inner skull) that can then be used for MEG (not EEG).

In [ ]:
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject=subject, ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

In [ ]:
bem_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-bem.fif')
mne.bem.write_bem_solution(bem_fname, bem)

In [ ]:
fwd = mne.make_forward_solution(raw_fname, trans=trans_fname,
                                src=src, bem=bem,
                                meg=True, # include MEG channels
                                eeg=False, # include EEG channels
                                mindist=5.0, # ignore sources <= 5mm from inner skull
                                n_jobs=1) # number of jobs to run in parallel

In [ ]:
fwd_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-meg-fwd.fif')
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

Or read the EEG/MEG file from disk

In [ ]:
fwd = mne.read_forward_solution(fwd_fname)

In [ ]:
fwd

Convert to surface orientation for cortically constrained inverse modeling:

In [ ]:
fwd = mne.convert_forward_solution(fwd, surf_ori=True)
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

Compute sensitivity maps for gradiometers

In [ ]:
sens_map = mne.sensitivity_map(fwd, ch_type='grad', mode='fixed')

# Show gain matrix a.k.a. leadfield matrix with sensitivy map

In [ ]:
import matplotlib.pyplot as plt

picks = mne.pick_types(fwd['info'], meg='grad', eeg=False)

im = plt.imshow(leadfield[picks, :500], origin='lower', aspect='auto', cmap='RdBu_r')
plt.xlabel('sources')
plt.ylabel('sensors')
plt.title('Lead field matrix for Gradiometers', fontsize=14)
plt.colorbar(cmap='RdBu_r')

In [ ]:
plt.figure()
plt.hist(sens_map.data.ravel(), bins=20, label='Gradiometers',
          color='c')
plt.legend()
plt.title('Normal orientation sensitivity')
plt.xlabel('sensitivity')
plt.ylabel('count');

Let's view in 3D:

In [ ]:
# enable correct backend for 3d plotting
clim = dict(kind='percent', lims=(0.0, 50, 99), smoothing_steps=3)  # let's see single dipoles
brain = sens_map.plot(subject=subject, time_label='GRAD sensitivity',
                      subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
view = 'lat'
brain.show_view(view)
brain.save_image('sensitivity_map_grad_%s.jpg' % view)
Image(filename='sensitivity_map_grad_%s.jpg' % view, width=400)

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Compute and plot the sensitivity maps for magnetometers and compare it with the gradiometers. </li>
      <li>Can you justify the claims that MEG is not sensitive to radial sources?</li>     
      <li>Try changing the `mode` parameter in `mne.sensitivity_map` and see how fixed or free orientation forward operator change the pictures. Comment.</li>
    </ul>
</div>